In [1]:
import pickle
import pandas as pd
#Rawdata.pkl is created in "Makedata.py"
with open("Rawdata.pkl", "rb") as load:
    X = pickle.load(load)

In [2]:
# Data preprocessing
# Drop empty fields
print(len(X))
X = X.dropna()
print(len(X))

10815072
10815004


In [3]:
#Lowercase everything
X['Text'] = X['Text'].astype(str)
X['Text'] = X['Text'].apply(lambda text: text.lower())

In [4]:
# Pre-make train and test data, avoid repetitions
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
X.reset_index(inplace=True, drop=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X['Text'], X['Lat'], test_size=0.1, random_state=100)

vectorizer = TfidfVectorizer(analyzer='word',lowercase=False, min_df=100)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print(X_train.shape)

(9733503, 51783)


In [6]:
from sklearn.decomposition import TruncatedSVD
reducer = TruncatedSVD(30)
X_train_trunc = reducer.fit_transform(X_train)
X_test_trunc = reducer.transform(X_test)

In [ ]:
from lightgbm.sklearn import LGBMRegressor

model = LGBMRegressor(n_jobs=-1, n_estimators=2500, learning_rate=0.35, num_leaves=9120)
model.fit(X_train_trunc, y_train)

from sklearn.metrics import accuracy_score
print(mean_absolute_error(y_test, model.predict(X_test_trunc)))